#### count Kerchunked JSON files 
Run this while the kerchunking is taking place to make sure list is growing

In [ ]:
import fsspec
import pandas as pd
import dask
import dask.bag as db

In [ ]:
import sys
import os
libDir = r'/shared/users/lib'
if libDir not in sys.path:
    sys.path.append(libDir)
    
import ebdpy as ebd
os.environ['AWS_PROFILE'] = 'esip-qhub'
client,cluster = ebd.start_dask_cluster(
    profile=os.environ['AWS_PROFILE'],
    worker_max=30,
    region='us-west-2', 
    use_existing_cluster=True,
    adaptive_scaling=False, 
    wait_for_cluster=False, 
    propagate_env=True)

In [ ]:
fs_json = fsspec.filesystem('s3', anon=False, skip_instance_cache=True)  

In [ ]:
json_dir = 's3://esip-qhub/noaa/nwm/grid1km/json/'

In [ ]:
%%time
jlist = fs_json.ls(json_dir, refresh=True)
print(len(jlist))
print(jlist[0])
print(jlist[-1])

In [ ]:
def get_size(json):
    return fs_json.size(json)

In [ ]:
b = db.from_sequence(jlist, npartitions=30)

In [ ]:
get_size(jlist[0])

In [ ]:
b1 = b.map(get_size)

In [ ]:
%%time
jsize = b1.compute(retries=10)

In [ ]:
df = pd.DataFrame({'file': jlist, 'size': jsize }).sort_values('size')

In [ ]:
list(df.iloc[:3]['file'].values)

In [ ]:
df.to_csv('json_sizes.csv')